In [4]:
import pandas as pd
from sqlalchemy import create_engine, text

# Configurações do banco de dados
DATABASE_TYPE = 'postgresql'
DBAPI = 'psycopg2'
USER = 'magazord_user'
PASSWORD = 'magazord'
HOST = 'localhost'
PORT = '5432'
DATABASE = 'magazord'

# Criação da string de conexão
connection_string = f"{DATABASE_TYPE}+{DBAPI}://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}"
engine = create_engine(connection_string)

# Função para verificar se a tabela transacoes está particionada corretamente
def verificar_particionamento():
    try:
        with engine.connect() as connection:
            # Verificar se a tabela 'transacoes' está particionada
            result = connection.execute(text("""
                SELECT to_regclass('transacoes');
            """))
            tabela_exists = result.fetchone()[0]

            if tabela_exists:
                result = connection.execute(text("""
                    SELECT relispartition
                    FROM pg_class
                    WHERE relname = 'transacoes';
                """))
                row = result.fetchone()

                if row and row[0]:
                    print("A tabela 'transacoes' está particionada.")
                else:
                    print("A tabela 'transacoes' não é particionada.")
            else:
                print("A tabela 'transacoes' não existe.")
    except Exception as e:
        print(f"Erro ao verificar particionamento: {e}")

# Função para verificar se os índices necessários existem para a tabela
def verificar_indices(tabela):
    try:
        with engine.connect() as connection:
            print(f"Verificando índices para a tabela {tabela}...")

            # Consultar os índices existentes para a tabela
            result = connection.execute(text(f"""
                SELECT indexname
                FROM pg_indexes
                WHERE tablename = '{tabela}';
            """))
            indices = [row[0] for row in result.fetchall()]

            # Índices esperados
            if tabela == 'clientes':
                indices_esperados = ['idx_id_cliente', 'idx_nome_cliente']
            elif tabela == 'produtos':
                indices_esperados = ['idx_id_produto']
            elif tabela == 'transacoes':
                indices_esperados = ['idx_id_cliente', 'idx_id_produto', 'idx_data_transacao']
            elif tabela == 'total_por_cliente':
                indices_esperados = ['idx_id_cliente']

            # Verificar se todos os índices necessários existem
            for idx in indices_esperados:
                if idx in indices:
                    print(f"Índice '{idx}' encontrado para a tabela {tabela}.")
                else:
                    print(f"Índice '{idx}' NÃO encontrado para a tabela {tabela}.")

    except Exception as e:
        print(f"Erro ao verificar índices para a tabela {tabela}: {e}")

# Função para verificar se todas as tabelas foram otimizadas
def verificar_otimizacao():
    try:
        tabelas = ['clientes', 'produtos', 'transacoes', 'total_por_cliente']

        for tabela in tabelas:
            # Verificar particionamento para a tabela transacoes
            if tabela == 'transacoes':
                verificar_particionamento()

            # Verificar índices para cada tabela
            verificar_indices(tabela)

        print("Verificação de otimização concluída!")

    except Exception as e:
        print(f"Erro ao verificar otimização: {e}")

# Executar a verificação de otimização para todas as tabelas
verificar_otimizacao()


Verificando índices para a tabela clientes...
Índice 'idx_id_cliente' encontrado para a tabela clientes.
Índice 'idx_nome_cliente' encontrado para a tabela clientes.
Verificando índices para a tabela produtos...
Índice 'idx_id_produto' encontrado para a tabela produtos.
A tabela 'transacoes' não é particionada.
Verificando índices para a tabela transacoes...
Índice 'idx_id_cliente' NÃO encontrado para a tabela transacoes.
Índice 'idx_id_produto' NÃO encontrado para a tabela transacoes.
Índice 'idx_data_transacao' NÃO encontrado para a tabela transacoes.
Verificando índices para a tabela total_por_cliente...
Índice 'idx_id_cliente' NÃO encontrado para a tabela total_por_cliente.
Verificação de otimização concluída!
